In [199]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [200]:
pip install control

Note: you may need to restart the kernel to use updated packages.


In [201]:
import sympy as sp
import control

In [202]:
x1,x2,x3,x4,m1,m2,l,u,g=sp.symbols("x1 x2 x3 x4 m1 m2 l u g")

In [203]:
x=sp.Matrix([x1,x2,x3,x4])

In [204]:
m1=1
m2=0.1
g=9.8
l=1

In [222]:
f1=x2
f2=u-(m2*l*(g*sp.cos(x3)+(x4**2))*sp.sin(x3))/(4*(m1+m2)-2*m2*((sp.cos(x3))**2))
f3=x4
f4=2*(f2*sp.cos(x3)+g*sp.sin(x3))/l

In [223]:
f=sp.Matrix([f1,f2,f3,f4])

In [224]:
A=f.jacobian(x)
B=f.jacobian([u])

In [225]:
subs = {x1: 0, x2: 0, x3: 0, x4: 0, u: 0}
A_lin = A.subs(subs)
B_lin = B.subs(subs)

In [226]:
A_lin

Matrix([
[0, 1,                  0, 0],
[0, 0, -0.233333333333333, 0],
[0, 0,                  0, 1],
[0, 0,   19.1333333333333, 0]])

In [227]:
B_lin

Matrix([
[0],
[1],
[0],
[2]])

In [228]:
A_lin = np.array(A_lin).astype(float)
B_lin = np.array(B_lin).astype(float)

In [241]:
Q=np.diag([1,10,10,10])
R=np.array([[1]])

In [242]:
K,_,_=control.lqr(A_lin,B_lin,Q,R)

In [243]:
obs=np.array([0,0,1.2,0])

In [244]:
controller(K,obs)

(1, 33.42897451134534)

In [245]:
def controller(K,obs):
    x_ref=np.array([0,0,0,0])
    dx=obs-x_ref
    u=np.dot(K,dx)
    act=-1
    if u[0]<0:
        act=0
    else:
        act=1
    return act,u[0]

In [246]:
import gym
import matplotlib.pyplot as plt
from gym.wrappers import RecordVideo
import math

In [247]:
from gym.envs.classic_control.cartpole import CartPoleEnv
env_name = 'CartPole-v1'

class CustomCartPoleEnv(CartPoleEnv):
    
    def __init__(self):
        super().__init__()
        self.theta_threshold_radians = 48 * 2 * math.pi / 360

In [248]:
env = RecordVideo(gym.make("CartPole-v1"), "./mp4")
# env=RecordVideo(CustomCartPoleEnv(),"./mp4")
env.env.seed(1)     # seed for reproducibility
obs = env.env.reset()
for i in range(1000):
    env.render()
    act,val=controller(K,obs)
    abs_force = abs(float(np.clip(val, -10, 10)))
    env.env.force_mag = abs_force

    obs,_,done,_,=env.step(act)
    
    print(obs)
    print(done)
    if done:
        print(f'Terminated after {i+1} iterations.')
        break

env.close()


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /kaggle/working/mp4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and

[ 0.00208309 -0.14954773 -0.03468674  0.32611182]
False
[-0.00090786  0.04605047 -0.0281645   0.02269515]
False
[ 1.3144526e-05 -1.4865649e-01 -2.7710600e-02  3.0636048e-01]
False
[-0.00295999  0.04684915 -0.02158339  0.00506874]
False
[-0.002023   -0.14795673 -0.02148202  0.29086447]
False
[-0.00498214  0.04746484 -0.01566473 -0.00851548]
False
[-0.00403284 -0.14742899 -0.01583504  0.27918416]
False
[-0.00698142  0.04791523 -0.01025135 -0.01845074]
False
[-0.00602312 -0.14705822 -0.01062037  0.2709802 ]
False
[-0.00896428  0.04821366 -0.00520076 -0.02503341]
False
[-0.00800001 -0.14683332 -0.00570143  0.2660041 ]
False
[-0.01093667  0.04836954 -0.00038135 -0.02847165]
False
[-0.00996928 -0.14674695 -0.00095078  0.26409093]
False
[-0.01290422  0.04838857  0.00433104 -0.02889173]
False
[-0.01193645 -0.14679523  0.0037532   0.26515454]
False
[-0.01487235  0.04827296  0.00905629 -0.02634224]
False
[-0.0139069  -0.1469777   0.00852945  0.26918423]
False
[-0.01684645  0.04802151  0.01391313